## Assignment 2 Data Analysis using Pandas

This assignment will contain 2 question with details as below. The due date is October 23 (Sunday), 2022 23:59PM. Each late day will result in 20% loss of total points.

### Question 1 (20 points) Datacamp courses

Datacamp is the online learning platform for data science courses and modules. As the only collaborator in Portugal, this course provides free access to the students in Business Analytics as the complementary resource. For this assignment, you are expected to complete the following moduels and submit your completion badges:

- [Importing & Cleaning Data with Python](https://app.datacamp.com/learn/skill-tracks/importing-cleaning-data-with-python)
- [Data Manipulation with Python](https://app.datacamp.com/learn/skill-tracks/data-manipulation-with-python)

Each badge will account for 10 points. If you are confident with the knowledge, you may skip the videos and directly finish the challenges. You can submit the badge by the end of the October if you would need more time. 

### Question 2 (80 points) Celluloid ceiling

Wonder Woman             |  Captain Marvel
:-------------------------:|:-------------------------:
![wonderwoman](https://upload.wikimedia.org/wikipedia/en/4/4f/Wonder_Woman_1984_poster.png) | ![marvel](https://upload.wikimedia.org/wikipedia/pt/5/59/Captain_Marvel_%282018%29.jpg)

Women are involved in the film industry in all roles, including as film directors, actresses, cinematographers, film producers, film critics, and other film industry professions, though women have been underrepresented in all these positions. Studies found that women have always had a presence in film acting, but have consistently been underrepresented, and on average significantly less well paid. 

In 2015, Forbes reported that "...just 21 of the 100 top-grossing films of 2014 featured a female lead or co-lead, while only 28.1% of characters in 100 top-grossing films were female... This means it’s much rarer for women to get the sort of blockbuster role which would warrant the massive backend deals many male counterparts demand (Tom Cruise in Mission: Impossible or Robert Downey Jr. in Iron Man, for example)".

Also, Forbes' analysis of US acting salaries in 2013 determined that the "...men on Forbes’ list of top-paid actors for that year made 2½ times as much money as the top-paid actresses. That means that Hollywood's best-compensated actresses made just 40 cents for every dollar that the best-compensated men made. 


In this assignment, we want to examine whether and how women representation is lacking in the film industry. We will adopt The Bechdel test as a measure of the representation of women in the film industry. The test is named after the American cartoonist Alison Bechdel in whose 1985 comic strip Dykes to Watch Out For the test first appeared. **A movie is said to meet the Bechdel test  following three criteria: (1) it has to have at least two women in it, who (2) who talk to each other, about (3) something besides a man.**

We are going to obtain the data ourselves to perform the analysis. Specifically, we will retrieve the movie metadata from IMDB (Internet Movie Database), an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews. As of January 2020, IMDb has approximately 6.5 million titles (including episodes) and 10.4 million personalities in its database, as well as 83 million registered users.


The IMDb Top 250 is a list of the top rated 250 films, based on ratings by the registered users of the website using the methods described. We will focus on these famous movies in this analysis:

**Question 2.1** (25 points): We will retrieve the metadata of IMDb Top 250 movies from the [IMDb charts](https://www.imdb.com/chart/top/). For each movie on the list, we can scrape the following characteristics from the information page. For example, from the [page of top rated movie "The Shawshank Redemption"](https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=F4QFC0SVZN1HTDHCY3C0&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1), we want to extract the metadata about this movie as:
- IMDb id (0111161)
- Movie name (The Shawshank Redemption)  
- Year (1994) 
- Director (Frank Darabont)
- Stars (Tim Robbins, Morgan Freeman, Bob Gunton)
- Rating (9.3)
- Number of reviews (2.6M)
- Genres (Drama)
- Country (USA)
- Language (English)
- Budget (\$25,000,000)
- Gross box Office Revenue (\$28,884,504)

![imdb](https://mrfloris.com/files/images/imdb-top250-page-start.png)


After scraping the 250 movies, save the data as a dataframe ```imdb_top_movies```. 

Also, saving the dataframe to a local file ```imdb_top_movies.csv``` so that later you can load it without scraping the website twice.

Hint: You can get the links to these movies from the IMDb top chart page, and then scrape each movie page by sending the request to these links. At each movie page, the information requested are located at different sections. 

In [ ]:
import pandas as pd
import numpy as np
import requests
from lxml import etree as et
from bs4 import BeautifulSoup

response = requests.get('http://www.imdb.com/chart/top')
soup = BeautifulSoup(response.text, "html.parser")
movies = soup.select('td.titleColumn') 
link = [x.attrs.get('href') for x in soup.select('td.titleColumn a')] #Get all the imdb ids in format of "/title/imdbid"
movies_info = []

for index in range(0,250):
    #Attach to the imdb main url the link that got in the previous page
    url = 'https://www.imdb.com' + link[index]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    dom = et.HTML(str(soup))
    
    #Split the string "/title/imdbid" to give us the id to store on dataset
    imdb_id = link[index].split("/")[2][2:] 
    #Return a list with some information about the movie and we select the 1st element which is always the movie name
    movie_name = dom.xpath('//h1[@data-testid="hero-title-block__title"]/text()')[0] 
    #Return a list with some information about the movie and we select the 1st element which is always the movie year
    movie_year = dom.xpath('//a[@class="ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh"]/text()')[0]
    #Return a list with some information about the movie and we select the 1st element which is always the director name
    director_name = dom.xpath('//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]/text()')[0]
    #Return a list with all the movie actors, and the stars are always the first 3 actors
    stars_get = dom.xpath('//div[@data-testid="shoveler-items-container"]/div/div/a/text()')[0:3]
    #Move all the the stars that get to a single string splitting them with a comma
    stars = ', '.join(stars_get)
    #Return the imdb rating and we access the 1st element because it always gives us a list so we need to get the string inside of that lost
    rating = dom.xpath('//span[@class="sc-7ab21ed2-1 jGRxWM"]/text()')[0]
    reviews = dom.xpath('//div[@class="sc-7ab21ed2-3 dPVcnq"]/text()')[0]

    #Here I just pass the value returned to an int 
    if "M" in reviews:
        if "." in reviews:
            reviews = int(''.join(x for x in reviews if x.isdigit())) * 100000
        else:
            reviews = int(''.join(x for x in reviews if x.isdigit())) * 1000000        
    else:
        reviews = int(''.join(x for x in reviews if x.isdigit())) * 1000    


    genre_get = dom.xpath('//span[@class="ipc-chip__text"]/text()')
    #Split all the genres just like in the stars, with a comma
    genre = ', '.join(genre_get)
    country_get = dom.xpath('//li[@data-testid="title-details-origin"]/div/ul/li/a/text()')
    #Split the country with a comma
    country = ', '.join(country_get)
    language_get = dom.xpath('//li[@data-testid="title-details-languages"]/div/ul/li/a/text()')
    #Split Language with a comma
    language = ', '.join(language_get)

    #Since sometimes the movies don't have information about the budget we have to use a try catch in order to make sure
    #if it returns an error the budget doesn't exist
    try:
        budget_get = dom.xpath('//li[@data-testid="title-boxoffice-budget"]/div/ul/li/span/text()')[0]
    except:
        budget_get = np.nan
        budget = np.nan
    else:
        #Only store the budget in dollars currency
        if "$" in budget_get:
            budget = ''.join(x for x in budget_get if x.isdigit())
        else:
            budget = np.nan

    #Since sometimes the movies don't have information about the gross we have to use a try catch in order to make sure
    #if it returns an error the gross doesn't exist
    try:
        gross_get = dom.xpath('//li[@data-testid="title-boxoffice-cumulativeworldwidegross"]/div/ul/li/span/text()')[0]
    except:
        gross_get = np.nan
        gross = np.nan
    else:
        #Only store the gross in dollars currency
        if "$" in gross_get:
            gross = ''.join(x for x in gross_get if x.isdigit())
        else:
            gross = np.nan
    
    runtime_get = dom.xpath('//div[@class="ipc-metadata-list-item__content-container"]/text()')
    #Runtime return the time in format hh:mm so we need to convert it to minutes
    if len(runtime_get) == 3:  # 2h
        runtime = int(int(runtime_get[0])*60)
    else:
        runtime = int(runtime_get[0])*60 + int(runtime_get[4])

    data = {'imdb_id': imdb_id,
            'movie_name': movie_name,
            'movie_year': movie_year,
            'director_name': director_name,
            'stars': stars,
            'rating': rating,
            'number_reviews' : reviews,
            'genre': genre,
            'country': country,
            'language': language,
            'budget': budget,
            'gross_revenue': gross,
            'runtime_minutes': runtime}
            
    movies_info.append(data)

df = pd.DataFrame(movies_info)
df.to_csv('imdb_top_movies.csv',index=False)

In [32]:
df = pd.read_csv('imdb_top_movies.csv')
df

,imdb_id,movie_name,movie_year,director_name,stars,rating,number_reviews,genre,country,language,budget,gross_revenue,runtime_minutes
0,111161,Os Condenados de Shawshank,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton",9.3,2700000,Drama,United States,English,25000000.0,2.888450e+07,142
1,68646,O Padrinho,1972,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan",9.2,1800000,"Crime, Drama",United States,"English, Italian, Latin",6000000.0,2.503418e+08,175
2,468569,O Cavaleiro das Trevas,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart",9.0,2600000,"Action, Crime, Drama","United States, United Kingdom","English, Mandarin",185000000.0,1.006234e+09,152
3,71562,O Padrinho: Parte II,1974,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall",9.0,1300000,"Crime, Drama",United States,"English, Italian, Spanish, Latin, Sicilian",13000000.0,4.796192e+07,202
4,50083,Doze Homens em Fúria,1957,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam",9.0,783000,"Crime, Drama",United States,English,350000.0,9.550000e+02,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,71411,Dersu Uzala - A Águia da Estepe,1975,Akira Kurosawa,"Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov",8.2,30000,"Adventure, Biography, Drama","Soviet Union, Japan","Russian, Chinese",4000000.0,1.448000e+04,142
246,1454029,As Serviçais,2011,Tate Taylor,"Viola Davis, Emma Stone, Octavia Spencer",8.1,458000,Drama,"United States, India",English,25000000.0,2.166391e+08,146
247,83987,Gandhi,1982,Richard Attenborough,"Ben Kingsley, John Gielgud, Rohini Hattangadi",8.1,232000,"Biography, Drama","United Kingdom, India, United States, South Af...","English, Hindi",22000000.0,5.276789e+07,191
248,103639,Aladdin,1992,Ron Clements,"Scott Weinger, Robin Williams, Linda Larkin",8.0,417000,"Animation, Adventure, Comedy",United States,English,28000000.0,5.040502e+08,90


**Question 2.2** (5 points) If you group the movies by release years, show the number of movies at each decade in descending order.

In [72]:
#Create a copy to don't do changes on original DF
df_answer = df.copy()
#Add column with the decades
df_answer["decade"] = df_answer["movie_year"] - (df_answer["movie_year"]%10)
#Reset index so in this way is easier to sort by the decades
df_answer.sort_values(['decade'],ascending=False).groupby("decade").size().reset_index().sort_index(ascending=False).rename(columns = {0: "Number of Movies"})

,decade,Number of Movies
10,2020,6
9,2010,43
8,2000,48
7,1990,41
6,1980,27
5,1970,19
4,1960,18
3,1950,23
2,1940,12
1,1930,7


**Quesion 2.3** (5 points) Show the number of movies by the distribution of runtime at quartile (0-25%, 25-50%, 50-75%, 75-100%).

In [37]:
pd.qcut(df["runtime_minutes"].rank(method='first'), q=4,labels=["0%-25%", "25%-50%", "50%-75%","75%-100%"] ).value_counts(sort = False, ascending = True)

0%-25%      63
25%-50%     62
50%-75%     62
75%-100%    63
Name: runtime_minutes, dtype: int64

**Question 2.4** (5 points) Show the top 10 most popular actor/actresses in terms of number of movies they have starred. 

In [38]:
#Create a new DF with multiple rows that are splited by the stars
multi_starsdf = df["stars"].str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
multi_starsdf = pd.DataFrame({"stars":multi_starsdf})

#Join the new DF created to the original df which contains now 3 times the same movie but with different stars because they are splitted 
all_stars_df = df.drop(["stars"], axis=1).join(multi_starsdf).reset_index(drop=True)
#Do reset_index just to have a beautifull presentation of the results and changed the named of the column 
all_stars_df.groupby("stars").size().nlargest(10).reset_index().rename(columns={0: "NumberAppearences"})


,stars,NumberAppearences
0,Robert De Niro,9
1,Harrison Ford,6
2,Leonardo DiCaprio,6
3,Tom Hanks,6
4,Charles Chaplin,5
5,Christian Bale,5
6,Clint Eastwood,5
7,Al Pacino,4
8,Brad Pitt,4
9,Jack Nicholson,4


**Question 2.5** (5 points) Show the average ratings of movies across the genres and decades.

In [39]:
#Create a new DF with multiple rows that are splited by the genre
multi_genredf = df["genre"].str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
multi_genredf = pd.DataFrame({"genre":multi_genredf})

#Join the new DF created to the original df which contains now some some multiple rows with the same movie but with different genres because they are splitted 
all_genre_df = df.drop(["genre"], axis=1).join(multi_genredf).reset_index(drop=True)
#Create a column with the decade of the movie
all_genre_df["decade"] = all_genre_df["movie_year"]- (all_genre_df["movie_year"] % 10)

print(all_genre_df.groupby(["genre", "decade"]).agg(mean_rating = ("rating", "mean")).to_string())


                  mean_rating
genre     decade             
Action    1920       8.200000
          1950       8.450000
          1960       8.400000
          1970       8.600000
          1980       8.288889
          1990       8.375000
          2000       8.472727
          2010       8.276923
          2020       8.266667
Adventure 1920       8.200000
          1930       8.100000
          1940       8.200000
          1950       8.200000
          1960       8.400000
          1970       8.240000
          1980       8.371429
          1990       8.250000
          2000       8.366667
          2010       8.307692
          2020       8.200000
Animation 1980       8.300000
          1990       8.260000
          2000       8.222222
          2010       8.271429
Biography 1920       8.200000
          1960       8.200000
          1970       8.200000
          1980       8.225000
          1990       8.550000
          2000       8.185714
          2010       8.188889
          

**Question 2.6** (10 points) Now let's retrieve data from Bechdel Test Movie website [for each movie](https://bechdeltest.com/). You can send the requests to the API: https://bechdeltest.com/api/v1/doc#getMovieByImdbId. For example, for the movie The Shawshank Redemption (the IMDb id: 0111161), you can simply call: http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=0111161. 

Create a dataframe ```bechdel_imdb_top``` that merge the bechdel test info with the ```imdb_top_movies``` show how many top 250 movies are also in the bechdel test website.

In [9]:
from bs4 import BeautifulSoup
import pandas as pd
bechdel_imdb_info = pd.DataFrame()

#Similar for to the one on 1st question, iterating the list that contais the ids in order to get the url of each movie
for index in range(0,250):
    url = 'http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(link[index].split("/")[2][2:])
    response = requests.get(url)
    todo = pd.read_json(response.text, typ = 'series')
    tododf = pd.DataFrame([todo])
    bechdel_imdb_info = pd.concat([bechdel_imdb_info, tododf], axis= 0)

bechdel_imdb_info.to_csv('bechdel_imdb_top.csv', index = False)


In [40]:
#Rename the column so they have the same name on both DF
df.rename(columns={"imdb_id": "imdbid"}, inplace=True)

#Convert all imbdid to int
df["imdbid"] = df["imdbid"].astype(int)

bechdel_imdb_info.dropna(subset = ["imdbid"], inplace = True)
#Convert all imbdid to int
bechdel_imdb_info["imdbid"] = bechdel_imdb_info["imdbid"].astype(int)

#Merge the common movies between df and bechdel_imdb_info with inner join
bechdel_imdb_top = pd.merge(df, bechdel_imdb_info, how='inner',on = ['imdbid'] )
#Rename columns so it becomes more easy to understand
bechdel_imdb_top.rename(columns={"rating_y": "rating_bechdel", "rating_x": "rating_imdb"}, inplace= True)

print("From the top 250 movies in imbdb,", len(bechdel_imdb_top.index), "are on Bechdel database")


From the top 250 movies in imbdb, 241 are on Bechdel database


**Question 2.7** (5 points) Show the percenage of movies given differen genres that has passed the test in different ways (Number from 0 to 3 (0 means no two women, 1 means no talking, 2 means talking about a man, 3 means it passes the test)) List the top 5 genres that has the highest fail rate (i.e., the movie passes 0 test)

In [41]:
#Create a new DF with multiple rows that are splited by the genre
multi_genredf2 = bechdel_imdb_top["genre"].str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
multi_genredf2 = pd.DataFrame({"genre":multi_genredf2})

#Join the new DF created to the original bechdel_df which contains now some some multiple rows with the same movie but with different genres because they are splitted 
all_genre_df2 = bechdel_imdb_top.drop(["genre"], axis=1).join(multi_genredf2).reset_index(drop=True)
#Use normalize to get the frequencies of each rating pear genre
movies_percentageDF = round(all_genre_df2.groupby(["genre"])["rating_bechdel"].value_counts(normalize = True, sort= False) * 100,2)

#Print of the 1st part of the question which return the percentage of bechdel rating per genre
print("Percentage of bechdel rating per genre\n",movies_percentageDF.to_string(), "\n")

#Print of the 2nd part of the question which return the top 5 genres with the highest bechdel rating = 0
print("Top 5 genres that have the highest fail rate\n",movies_percentageDF.swaplevel()[0.0].nlargest(5))


Percentage of bechdel rating per genre
 genre      rating_bechdel
Action     0.0                19.61
           1.0                29.41
           2.0                 1.96
           3.0                49.02
Adventure  0.0                21.67
           1.0                33.33
           2.0                 3.33
           3.0                41.67
Animation  0.0                13.04
           1.0                43.48
           3.0                43.48
Biography  0.0                14.81
           1.0                37.04
           2.0                14.81
           3.0                33.33
Comedy     0.0                21.74
           1.0                34.78
           2.0                 8.70
           3.0                34.78
Crime      0.0                16.33
           1.0                36.73
           2.0                16.33
           3.0                30.61
Drama      0.0                20.36
           1.0                32.93
           2.0                12.5

**Question 2.8** (5 points) Show the top 10 highest-rated English and non-English movies that passed the test completely (rating=3), respectively. 

In [48]:
rating_3 = bechdel_imdb_top.loc[(bechdel_imdb_top["rating_bechdel"] == 3)]
lang_eng_eng = rating_3[rating_3['language'] == ("English")]
lang_eng = rating_3[rating_3['language'].str.contains("English")]
lang_not_eng = rating_3[~rating_3['language'].str.contains("English")]

print("10 Highest-Rated Only English movies\n",lang_eng_eng.nlargest(10,"rating_imdb")[["movie_name", "rating_imdb"]], "\n")
print("10 Highest-Rated English movies\n",lang_eng.nlargest(10,"rating_imdb")[["movie_name", "rating_imdb"]], "\n")
print("10 Highest-Rated non-English movies\n",lang_not_eng.nlargest(10,"rating_imdb")[["movie_name", "rating_imdb"]])

10 Highest-Rated Only English movies
                         movie_name  rating_imdb
15                          Matrix          8.7
25                    Interstellar          8.6
31                           Psico          8.5
48               Janela Indiscreta          8.5
50        Alien - O 8.º Passageiro          8.5
58           Crepúsculo dos Deuses          8.4
63  Vingadores: Guerra do Infinito          8.4
65                Beleza Americana          8.4
70                           Joker          8.4
77               Top Gun: Maverick          8.4 

10 Highest-Rated English movies
                              movie_name  rating_imdb
2                O Cavaleiro das Trevas          9.0
5                  A Lista de Schindler          9.0
7                          Pulp Fiction          8.9
12                             A Origem          8.8
13  O Senhor dos Anéis - As Duas Torres          8.8
15                               Matrix          8.7
16                    Tudo B

**Question 2.9** (5 points) Creat a new column ROI that measures the return of investment using (box revenue-budget)/budget, and compare the average ROI between movies that passed (rating=3) and failed the test (rating=0), are their ROI different? Explain.

In [43]:
bechdel_imdb_top["ROI"] = (bechdel_imdb_top["gross_revenue"]-bechdel_imdb_top["budget"]) / bechdel_imdb_top["budget"]
rating_3 = bechdel_imdb_top.loc[(bechdel_imdb_top["rating_bechdel"] == 3)]
rating_0 = bechdel_imdb_top.loc[(bechdel_imdb_top["rating_bechdel"] == 0)]

print("Average ROI on movies with rating 3 is",rating_3["ROI"].mean())
print("Average ROI on movies with rating 0 is",rating_0["ROI"].mean())


Average ROI on movies with rating 3 is 10.218064889520797
Average ROI on movies with rating 0 is 5.6888893828004115


The ROIs obtained are different, the one with the rating 3 is arround 10.2 and the rating 0 is arround 5.68

**Question 2.10** (10 points) Now load the bech test results from all movies (https://bechdeltest.com/api/v1/doc#getAllMovies) that contains the all movies that are rated by the Bechdel Test website. Are women representation improved over the decades? Create a dataframe ```bechdel_imdb```, comparing the top 250 and other movies, in terms of percentage, how many passed/failed the test? 

In [76]:
response = requests.get('http://bechdeltest.com/api/v1/getAllMovies')
new_json = pd.read_json(response.text, typ = 'series')
df_all_movies = pd.DataFrame()
for i in new_json:
    df_aux = pd.DataFrame([i])
    df_all_movies = pd.concat([df_all_movies,df_aux], axis= 0)


In [78]:
df_all_movies["decade"] = df_all_movies["year"] - (df_all_movies["year"]%10)

answer_df2 = df_all_movies.groupby(["decade"])["rating"].value_counts(normalize = True, sort= False) * 100
print(answer_df2.to_string())
print("In this qustion I am assuming that the women representation in a movie is when the rating in different",
     "of 0, which means that I am calculating the mean percentage of the Movies with the rating = 1,2, or 3")
print("By the results obtained we can see that the rating = 0 decreased over the decades followed by and increasement of the rating = 1,2, or 3\n")

                                            #########Second Part of the exercise#########

#Copy the original DF to don't make changes on that
bechdel_copy = bechdel_imdb_top.copy()

#Create a column on DF with yes meaning that he is on top250 imdb
bechdel_copy["TOP250"] = "YES"
#There are 4 movies without imdbid so I'm droping those movies
df_all_movies.loc[df_all_movies["imdbid"] == ""] = np.nan
df_all_movies.dropna(subset = ["imdbid"], inplace=True)

df_all_movies["imdbid"] = df_all_movies["imdbid"].astype(int)
#Merge the copy and the df that contain all movies by is id
bechdel_imdb = pd.merge(bechdel_copy , df_all_movies, how='outer',on = ['imdbid'])     
#Fill the nan values on column TOP250 with no meaning that the movie is not on the top250 
bechdel_imdb["TOP250"].fillna("NO", inplace=True)
#Drop the movies that have no rating
bechdel_imdb.dropna(subset=['rating'], inplace=True)

passpercentagetop250 = (len(bechdel_imdb[(bechdel_imdb["rating"] == 3) & (bechdel_imdb["TOP250"] == "YES")]) /  len(bechdel_imdb[bechdel_imdb["TOP250"] == "YES"]))*100
passpercentageothermovies = (len(bechdel_imdb[(bechdel_imdb["rating"] == 3) & (bechdel_imdb["TOP250"] == "NO")]) /  len(bechdel_imdb[bechdel_imdb["TOP250"] == "NO"]))*100
print("Pass percentage of top 250 movies on IMDB is:", round(passpercentagetop250,2), "%")
print("Fail percentage of top 250 movies on IMDB is:", round(100 - passpercentagetop250,2), "%")
print("Pass percentage of other movies (not on IMDB top 250) is:", round(passpercentageothermovies,2), "%")
print("Fail percentage of other movies (not on IMDB top 250) is:", round(100 - passpercentageothermovies,2),"%")



decade  rating
1870.0  0.0       100.000000
1880.0  0.0       100.000000
1890.0  0.0        95.522388
        1.0         2.985075
        3.0         1.492537
1900.0  0.0        92.857143
        1.0         4.761905
        2.0         2.380952
1910.0  0.0        42.307692
        2.0        15.384615
        3.0        42.307692
1920.0  0.0        43.373494
        1.0        13.253012
        2.0        22.891566
        3.0        20.481928
1930.0  0.0        10.837438
        1.0        19.704433
        2.0        20.689655
        3.0        48.768473
1940.0  0.0        14.937759
        1.0        20.746888
        2.0        14.937759
        3.0        49.377593
1950.0  0.0        14.334471
        1.0        21.843003
        2.0        13.993174
        3.0        49.829352
1960.0  0.0        20.104439
        1.0        26.631854
        2.0        10.182768
        3.0        43.080940
1970.0  0.0        14.285714
        1.0        27.891156
        2.0        11.791383